In [1]:
import datetime
import pandas as pd

In [2]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

In [3]:
%run "C:/Users/prasannanjaneyulu.b/OneDrive - Landmark Group india/Stats_Strats/seleting_instruments.ipynb"

%run "C:/Users/prasannanjaneyulu.b/OneDrive - Landmark Group india/Stats_Strats/Login_and_Mail_Alerts.ipynb"

%run "C:/Users/prasannanjaneyulu.b/OneDrive - Landmark Group india/Stats_Strats/Basket_and_direction_widgets.ipynb"

## LOGGING IN

In [4]:
kite = kite_login(api_key="kvvzk6jq4zn4o4vc",    api_secret="nzcqxwu05jmo2k71cmpgnbf5trz6gypn")

HTML(value='')

In [5]:
profile = kite.profile()

# Step 3: Print profile information
print("User ID:", profile["user_id"])
print("User Name:", profile["user_name"])
print("Email:", profile["email"])
print("User type:", profile["user_type"])
print("Products:", profile["products"])
print("Broker info:", profile["broker"])

User ID: FVC741
User Name: Balam Baji Babu
Email: balambajibabu143@gmail.com
User type: individual/res_no_nn
Products: ['CNC', 'NRML', 'MIS', 'BO', 'CO']
Broker info: ZERODHA


## CHART MONITOR

In [6]:
instruments = kite.instruments()

In [7]:
display(toggle)

ToggleButton(value=False, button_style='danger', description='Monitor OFF', icon='stop', tooltip='Toggle candl…

## PICKING DIRECTION

In [8]:
from datetime import datetime

weekday = datetime.today().weekday()

lp, up = (50, 70) if weekday in (0, 1) else (40, 60)
fut_symbols, option_symbols, tokens = pick_direction(kite, instruments, lp=lp, up=up)


# MAKING ORDER BASKET

In [10]:
make_order_basket(fut_symbols, option_symbols, kite)

Output()

Output()

Output()

Output()

## Placing Normal Orders

In [11]:
"YESBANK"

'YESBANK'

In [12]:
place_order(kite)

Output()

Output()

Output()

Output()

# Exit Orders and Placing Stop Loss Order for Spread Leg

In [13]:
exit_position_manager(kite)

Output()

Output()

Output()

Output()

Output()

Output()

## Placing Stop Loss Order for Spread Leg

## Modifying Orders

In [14]:
position_exit_manager_with_trail_sl_and_ticker_2222(kite, api_key = kite.api_key)

Output()

Output()

Output()

Output()

Output()

Output()